In [1]:
!git clone https://github.com/djwackey/chinese-hiphop-lyrics.git
import os,json

allhiphop = os.walk('./chinese-hiphop-lyrics/')
songs = []
for root_dir,_,hiphops in allhiphop:
    for hiphop in hiphops:
       if hiphop.endswith(".json"):
        song_path = f'{root_dir}/{hiphop}'
        with open(song_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
                for song in data['songs']:
                    songs.append(song)

fatal: destination path 'chinese-hiphop-lyrics' already exists and is not an empty directory.


In [2]:

file_path = 'word.txt'
for song in songs:
    # print(song)
    text = song['lyrics']
    lines = 'EOS'.join(text)
    with open(file_path, 'a', encoding='utf-8') as file:
        file.write(lines)


## Markov Chain Model

In [3]:
import random
import jieba  # Used for Chinese word segmentation

def read_file(file_path):
    # Read and return text from the file
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

def preprocess_text(text):
    # Segment the text into words using jieba and return the list of words
    words = list(jieba.cut(text))
    return words

def build_markov_chain(words):
    # Build a Markov chain dictionary where each word points to a list of words that come after it
    markov_chain = {}
    for i in range(len(words) - 1):
        word = words[i]
        next_word = words[i + 1]
        if word not in markov_chain:
            markov_chain[word] = []
        markov_chain[word].append(next_word)
    return markov_chain

def generate_text(chain, length=20):
    # Generate a text of specified length using the Markov chain
    word1 = random.choice(list(chain.keys()))
    result = [word1]
    for _ in range(length - 1):
        word2 = random.choice(chain[word1])
        result.append(word2)
        word1 = word2
    return ''.join(result)

file_path = './word.txt'  # Replace with your file path
text = read_file(file_path)  # Read the text from the file
words = preprocess_text(text)  # Preprocess the text for Markov chain
chain = build_markov_chain(words)  # Build the Markov chain
generated_text = generate_text(chain, 50)  # Generate text of length 50
print(generated_text)  # Print the generated text

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.cache
Loading model cost 0.805 seconds.
DEBUG:jieba:Loading model cost 0.805 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


westside
超脱后悔过过招像日本出品嘿 世世你的路EOS在梦EOS加入我们都为她吃
以前我制造 导步EOS人生被雪覆盖整片海域因为我的路EOS谁会跌倒的 can 


In [4]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Read text data
file_path = 'word.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

# Tokenize using jieba
words = list(jieba.cut(text))

# Build vocabulary
vocab = set(words)
word_to_index = {word: idx for idx, word in enumerate(vocab)}
index_to_word = {idx: word for idx, word in enumerate(vocab)}

# Convert text to index sequences
sequences = [word_to_index[word] for word in words]

# Create sequences and labels
sequence_length = 10  # Set the length of each sequence
sequences_input = []
labels = []

for i in range(len(sequences) - sequence_length):
    seq = sequences[i:i + sequence_length]
    label = sequences[i + sequence_length]
    sequences_input.append(seq)
    labels.append(label)

# Convert to PyTorch tensors
sequences_input = torch.tensor(sequences_input, dtype=torch.long)
labels = torch.tensor(labels, dtype=torch.long)

# Split into training and testing sets
split_ratio = 0.8  # 80% for training, 20% for testing
split_index = int(len(sequences_input) * split_ratio)

train_sequences = sequences_input[:split_index]
train_labels = labels[:split_index]

test_sequences = sequences_input[split_index:]
test_labels = labels[split_index:]

# Create data loaders
batch_size = 2048
train_dataset = TensorDataset(train_sequences, train_labels)
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(test_sequences, test_labels)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [5]:
label

2616

In [6]:
# 查看训练数据和测试数据的一个批次
for batch_inputs, batch_labels in train_data_loader:
    # print("Train Batch Inputs:", batch_inputs)
    print("Train Batch Labels:", batch_labels)
    break

for batch_inputs, batch_labels in test_data_loader:
    # print("Test Batch Inputs:", batch_inputs)
    print("Test Batch Labels:", batch_labels)
    break

Train Batch Labels: tensor([ 9867, 12631, 11812, 35445, 40129, 37345, 20891, 36478, 11976, 37050,
        15001, 72604, 37882, 11976, 37882, 65587, 11976, 76178, 32887, 58246,
        18688, 79590, 67757, 11976, 47600, 57095, 72697, 61719, 70323, 36247,
        43138, 42512, 54293,  4955, 35472, 37882, 72281, 32172, 45145, 23669,
        34088, 23066, 64239, 11976, 42299, 12631, 11976, 40428, 26270, 11976,
        11976, 62139, 39322, 67032,  3584, 42668, 42967, 21727, 21951, 47010,
        38576, 32019,  5731, 23890])
Test Batch Labels: tensor([32956, 11976, 64798, 11976, 59542, 11976, 32956, 11976, 31082, 11976,
        38242, 11976, 32956, 11976, 49725, 45333, 17764, 22665, 11976, 24118,
        11976, 20865, 11976, 43846, 17764, 12631, 22463, 72697,  3319, 71515,
        17764, 12631, 46343, 72697,  3319, 71515, 17764, 13011, 23433, 43696,
        72697,  3319, 71515, 17764, 68958, 67032,  2326, 15045, 11976, 67963,
        28434, 35703, 17764, 74316, 70005, 54693, 65863, 11976,  6

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

# Assume you already have processed data: 'train_sequences', 'train_labels', 'test_sequences', 'test_labels'
# Representing input sequences and labels for the training and testing sets

# LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        output, (hidden, cell) = self.lstm(x)
        output = self.fc(output[:, -1, :])
        return output

# Hyperparameters
vocab_size = len(index_to_word)  # Assuming vocabulary size is 10000
embedding_dim = 128  # Embedding dimension
hidden_dim = 256  # LSTM hidden layer dimension
output_dim = vocab_size  # Output dimension same as vocabulary size
sequence_length = 10  # Set the length of each sequence

# Instantiate the model
model = LSTMModel(vocab_size, embedding_dim, hidden_dim, output_dim)

# Define loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convert to PyTorch tensors
train_sequences = torch.tensor(train_sequences, dtype=torch.long)
train_labels = torch.tensor(train_labels, dtype=torch.long)

test_sequences = torch.tensor(test_sequences, dtype=torch.long)
test_labels = torch.tensor(test_labels, dtype=torch.long)

# Create data loaders
batch_size = 2048
train_dataset = TensorDataset(train_sequences, train_labels)
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(test_sequences, test_labels)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Training and validation
num_epochs = 3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

total_iterations = 0  # Iteration counter
for epoch in range(num_epochs):
    # Training phase
    model.train()
    total_loss = 0.0
    for batch_inputs, batch_labels in tqdm(train_data_loader, desc=f'Epoch {epoch + 1}/{num_epochs} - Training', position=0):
        total_iterations += 1
        batch_inputs, batch_labels = batch_inputs.to(device), batch_labels.to(device)
        optimizer.zero_grad()
        outputs = model(batch_inputs)
        loss = loss_function(outputs, batch_labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # Output training loss and perplexity every 100 iterations
        if total_iterations % 100 == 0:
            avg_loss = total_loss / 100
            perplexity = torch.exp(torch.tensor(avg_loss))
            tqdm.write(f'Iteration {total_iterations}, Training Loss: {avg_loss:.4f}, Perplexity: {perplexity:.4f}')
            total_loss = 0.0

    # Validation phase
    model.eval()
    with torch.no_grad():
        total_correct = 0
        total_samples = 0
        for batch_inputs, batch_labels in tqdm(test_data_loader, desc=f'Epoch {epoch + 1}/{num_epochs} - Validation'):
            batch_inputs, batch_labels = batch_inputs.to(device), batch_labels.to(device)
            outputs = model(batch_inputs)
            _, predicted = torch.max(outputs, 1)
            total_correct += (predicted == batch_labels).sum().item()
            total_samples += batch_labels.size(0)

        accuracy = total_correct / total_samples
        tqdm.write(f'Validation Accuracy: {accuracy * 100:.2f}%')

print('Training complete.')


<ipython-input-7-bc6874187399>:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_sequences = torch.tensor(train_sequences, dtype=torch.long)
<ipython-input-7-bc6874187399>:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels = torch.tensor(train_labels, dtype=torch.long)
<ipython-input-7-bc6874187399>:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_sequences = torch.tensor(test_sequences, dtype=torch.long)
<ipython-input-7-bc6874187399>:43: UserWarning: To copy construct from a tensor, it is recommended to use sourc

Iteration 100, Training Loss: 7.5969, Perplexity: 1991.9426


Epoch 1/3 - Training:   6%|▌         | 202/3625 [00:12<02:59, 19.12it/s]

Iteration 200, Training Loss: 6.6152, Perplexity: 746.3846


Epoch 1/3 - Training:   8%|▊         | 303/3625 [00:18<05:09, 10.74it/s]

...

Iteration 10700, Training Loss: 2.8229, Perplexity: 16.8255


Epoch 3/3 - Training:  98%|█████████▊| 3553/3625 [04:55<00:06, 10.67it/s]

Iteration 10800, Training Loss: 2.8127, Perplexity: 16.6546


Epoch 3/3 - Validation: 100%|██████████| 907/907 [00:48<00:00, 18.62it/s]

Validation Accuracy: 49.74%
Training complete.


In [8]:
vocab = list(vocab)

In [9]:
vocab[0]

'下论'

In [10]:
torch.save(model.state_dict(), 'model.pth')

In [23]:
import torch
import random
from torch.nn import functional as F

model.eval()

org = [
    "唉哟 P.Q我来自上海的兄弟",
    "What’s up",
     "我的新疆homie",
     "Are you ready",
     "I’m ready",
    "Let’s go",
    "初次去上海只为见心爱的她",
    "南京路上相拥",
    "听跨年钟声嘀嗒",
    "忍不住狂甩对方嘴唇在那一刹",
    "更幸福的是",
    "现在看我们孩子长大",
    "街上都是迪丽热巴和古力娜扎",
    "在乌鲁木齐每一天的太阳",
    "So hot",
    "忘不了那抓饭的香和大盘鸡的辣",
     "我喝了十六次的石榴汁在大巴扎",
     "那年上海夺了冠",
     "狗哥微博发过赞",
     "P.Q带我吃的串",
    "象哥的酒量赢一半"
]

# Function to convert generated indices back to words
def index_to_word(index, vocab):
    return vocab[index]

# Function to generate sentences
def generate_sentence(model, initial_word, vocab, max_length=5):
    model.eval()
    # model.to('cpu')
    # Convert the initial word to an index
    current_word = torch.tensor([word_to_index[initial_word]], dtype=torch.long).unsqueeze(0).to(device)

    generated_sentence = [initial_word]

    with torch.no_grad():
        for _ in range(max_length):
            output = model(current_word)
            probabilities = F.softmax(output, dim=1).squeeze(0)
            predicted_index = torch.multinomial(probabilities, 1).item()
            predicted_word = index_to_word(predicted_index, vocab)

            if 'EOS' in predicted_word:
                break

            generated_sentence.append(predicted_word)
            current_word = torch.tensor([predicted_index], dtype=torch.long).unsqueeze(0).to(device)

    return ' '.join(generated_sentence)

# Test sentence generation
generated_sentences = []
for org_sentence in org:
    initial_word = org_sentence[0]  # Use the first character of each sentence as the initial word
    generated_sentence = generate_sentence(model, initial_word, vocab)
    generated_sentences.append(generated_sentence)

# Print the generated sentences
for org_sentence, gen_sentence in zip(org, generated_sentences):
    print(f"Original: {org_sentence}")
    print(f"Generated: {gen_sentence}")
    print("-" * 50)


Original: 唉哟 P.Q我来自上海的兄弟
Generated: 唉 千奇百怪 相处 阴晴圆 浓郁 Whiskey
--------------------------------------------------
Original: What’s up
Generated: W koralmaysan
--------------------------------------------------
Original: 我的新疆homie
Generated: 我 握紧 卡卡 先人 四合
--------------------------------------------------
Original: Are you ready
Generated: A 喷气式 我梦里 思绪 太过火
--------------------------------------------------
Original: I’m ready
Generated: I 胡言
--------------------------------------------------
Original: Let’s go
Generated: L 笨重
--------------------------------------------------
Original: 初次去上海只为见心爱的她
Generated: 初 脑洞 鲜 静静的 Do GogoBoi
--------------------------------------------------
Original: 南京路上相拥
Generated: 南 走險 里路 淫荡 照料 视频
--------------------------------------------------
Original: 听跨年钟声嘀嗒
Generated: 听 牺牲品 走私犯 fendi 盘膝 全班
--------------------------------------------------
Original: 忍不住狂甩对方嘴唇在那一刹
Generated: 忍 rolling 摧残 DIZZY 除病 不择手段
--------------------------------------------------
Or

In [25]:
import nltk
from nltk.translate.bleu_score import sentence_bleu

# Tokenize the original and generated sentences
tokenized_org = [nltk.word_tokenize(sentence.lower()) for sentence in org]
tokenized_gen = [nltk.word_tokenize(sentence.lower()) for sentence in generated_sentences]

# Calculate BLEU score with lower n-gram order (e.g., 1-gram)
bleu_scores = [sentence_bleu([tokens], generated_tokens, weights=(4, 0, 0, 0)) for tokens, generated_tokens in zip(tokenized_org, tokenized_gen)]

# Print BLEU scores
for org_sentence, gen_sentence, bleu_score in zip(org, generated_sentences, bleu_scores):
    print(f"Original: {org_sentence}")
    print(f"Generated: {gen_sentence}")
    print(f"BLEU Score: {bleu_score:.4f}")
    print("-" * 50)


Original: 唉哟 P.Q我来自上海的兄弟
Generated: 唉 千奇百怪 相处 阴晴圆 浓郁 Whiskey
BLEU Score: 0.0000
--------------------------------------------------
Original: What’s up
Generated: W koralmaysan
BLEU Score: 0.0000
--------------------------------------------------
Original: 我的新疆homie
Generated: 我 握紧 卡卡 先人 四合
BLEU Score: 0.0000
--------------------------------------------------
Original: Are you ready
Generated: A 喷气式 我梦里 思绪 太过火
BLEU Score: 0.0000
--------------------------------------------------
Original: I’m ready
Generated: I 胡言
BLEU Score: 0.0230
--------------------------------------------------
Original: Let’s go
Generated: L 笨重
BLEU Score: 0.0000
--------------------------------------------------
Original: 初次去上海只为见心爱的她
Generated: 初 脑洞 鲜 静静的 Do GogoBoi
BLEU Score: 0.0000
--------------------------------------------------
Original: 南京路上相拥
Generated: 南 走險 里路 淫荡 照料 视频
BLEU Score: 0.0000
--------------------------------------------------
Original: 听跨年钟声嘀嗒
Generated: 听 牺牲品 走私犯 fendi 盘膝 全班
BLEU Score: 0.

In [22]:
tokenized_org

[['唉哟', 'p.q我来自上海的兄弟'],
 ['what', '’', 's', 'up'],
 ['我的新疆homie'],
 ['are', 'you', 'ready'],
 ['i', '’', 'm', 'ready'],
 ['let', '’', 's', 'go'],
 ['初次去上海只为见心爱的她'],
 ['南京路上相拥'],
 ['听跨年钟声嘀嗒'],
 ['忍不住狂甩对方嘴唇在那一刹'],
 ['更幸福的是'],
 ['现在看我们孩子长大'],
 ['街上都是迪丽热巴和古力娜扎'],
 ['在乌鲁木齐每一天的太阳'],
 ['so', 'hot'],
 ['忘不了那抓饭的香和大盘鸡的辣'],
 ['我喝了十六次的石榴汁在大巴扎'],
 ['那年上海夺了冠'],
 ['狗哥微博发过赞'],
 ['p.q带我吃的串'],
 ['象哥的酒量赢一半']]

In [12]:
from sklearn.model_selection import train_test_split

# 假设 X_padded 和 y 是你的完整数据集
# X_padded: 输入序列
# y: 目标标签

# 设置切分比例，例如 80% 训练数据，20% 验证数据
train_size = 0.8

# 切分数据
X_train, X_val, y_train, y_val = train_test_split(X_padded, y, train_size=train_size, random_state=42)

# 打印切分后数据的大小
print("训练集大小:", X_train.shape)
print("验证集大小:", X_val.shape)


NameError: ignored

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

class LyricsDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx]

# LSTM 模型
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        output, (hidden, cell) = self.lstm(x)
        output = self.fc(output[:, -1, :])
        return output

In [ ]:
vocab_size = len(word_to_index)
embedding_dim = 512  # 嵌入维度
hidden_dim = 256  # LSTM 隐藏层维度
output_dim = vocab_size  # 输出维度与词汇表大小相同

In [ ]:
vocab_size

In [ ]:
# 准备数据加载器
batch_size = 1024
# 准备数据加载器
train_dataset = LyricsDataset(X_train, y_train)
val_dataset = LyricsDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# 实例化模型
model = LSTMModel(vocab_size+1, embedding_dim, hidden_dim, output_dim)
model.to(device)

# 定义损失函数和优化器
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10

# 训练和评估模型
def evaluate_model(model, data_loader, loss_function):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = loss_function(outputs, targets)
            total_loss += loss.item()
    return total_loss / len(data_loader)

# from tqdm import tqdm

# 在训练循环外部初始化 tqdm 进度条
# epoch_bar = tqdm(range(num_epochs), desc="Epochs")

from tqdm import tqdm
# epoch_bar = tqdm(range(num_epochs), total=num_epochs,desc="Epochs", position=0)
# 在训练循环外部初始化 tqdm 进度条
for epoch in range(num_epochs):
    model.train()

    # 在 train_loader 外部初始化 tqdm 进度条
    train_iterator = tqdm(enumerate(train_loader), total=len(train_loader), desc="Training Iterations", position=0)

    for i,(inputs, targets) in train_iterator:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            # 打印信息的部分保留，如果需要的话
            train_iterator.set_postfix({'desc':f"Epoch {epoch+1}, Iteration {i}, Train Loss: {loss},  Perplexity: {np.exp(loss.item())}"})

    # 关闭 train_iterator 进度条
#     train_iterator.close()

    # 在验证集上评估模型
    # train_loss = evaluate_model(model, train_loader, loss_function)
    val_loss = evaluate_model(model, val_loader, loss_function)

    # 打印信息的部分保留，如果需要的话
    print(f"Epoch {epoch+1}/{num_epochs}, Val Loss: {val_loss} ,Perplexity: {np.exp(val_loss)}")

In [ ]:
!pip install tqdm


In [ ]:
text[:30]

### LSTM

In [ ]:
import jieba
import torch
import numpy as np
from collections import Counter
from torch.utils.data import Dataset, DataLoader

def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

def preprocess_text(text):
    words = list(jieba.cut(text))
    return words

def create_vocab(words):
    word_counts = Counter(words)
    vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    word_to_idx = {word: idx for idx, word in enumerate(vocab)}
    idx_to_word = {idx: word for word, idx in word_to_idx.items()}
    return word_to_idx, idx_to_word

def create_sequences(words, word_to_idx, seq_length=20):
    sequences = []
    labels = []
    for i in range(len(words) - seq_length):
        seq = words[i:i+seq_length]
        label = words[i+seq_length]
        sequences.append([word_to_idx[word] for word in seq])
        labels.append(word_to_idx[label])
    return sequences, labels

# 读取和预处理文本
file_path = 'word.txt'  # 替换为你的文件路径
text = read_file(file_path)
words = preprocess_text(text)

# 创建词汇表和索引
word_to_idx, idx_to_word = create_vocab(words)

# 创建序列和标签
seq_length = 20  # 可根据需要调整序列长度
sequences, labels = create_sequences(words, word_to_idx, seq_length)

# 将序列和标签转换为 PyTorch 张量
sequences = torch.tensor(sequences, dtype=torch.long)
labels = torch.tensor(labels, dtype=torch.long)


In [ ]:
labels,sequences

In [ ]:
vocab_size = len(idx_to_word)
print("Vocabulary size:", vocab_size)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

# 假设您已经有了处理过的数据：'sequences' 和 'labels'
# sequences: 输入序列的集合
# labels: 每个序列的下一个词的标签

class LyricsDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx]

# LSTM 模型
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        output, (hidden, cell) = self.lstm(x)
        output = self.fc(output[:, -1, :])
        return output

# 设置超参数
# vocab_size = vocab_size  # 假设词汇表大小为 10000
embedding_dim = 128  # 嵌入维度
hidden_dim = 256  # LSTM 隐藏层维度
output_dim = vocab_size  # 输出维度与词汇表大小相同

# 实例化模型
model = LSTMModel(vocab_size, embedding_dim, hidden_dim, output_dim)

# 定义损失函数和优化器
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 准备数据加载器
batch_size = 64
dataset = LyricsDataset(sequences, labels)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 训练模型
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, targets in data_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")
